In [ ]:
%matplotlib tk
import pyxem as pxm
import hyperspy.api as hs
import numpy as np
import os

# Au Calibration

In [ ]:
filenames = []
for root, dirs, files in os.walk("/home/hremadmin/Documents/Pharma_EM17997/day5/AuCalibrationSunday/"):
    for file in files:
        if file.endswith(".mib"):
             filenames.append(os.path.join(root, file))

In [ ]:
filenames[-1]

In [ ]:
n_images = 256*256

def load_mib_scan_data(fname, dtype, n_images):
    offset = 192
    #read the binary data from file
    raw_image = np.fromfile(fname, dtype = dtype)
    #define empty numpy array into which the data arranged
    raw_image2 = np.zeros((256,256, n_images), dtype=dtype)
    #add each frame into the data array
    for n in np.arange(n_images):
        temp_slice = raw_image[n*(256**2) + (n+1) * offset :(n+1)*(256**2) + (n+1) * offset]
        raw_image2[:,:,n] = np.reshape(temp_slice,(256,256))
    raw_image = None
    #convert data stack to HyperSpy Signal2D
    dat = hs.signals.Signal2D(raw_image2.reshape(256,256,256,256)).T
    #sum the dffraction and slow scan dimensions to obtain line trace for flyback edge
    trace = dat.sum((1,2,3))
    edge = np.where(trace==max(trace.data))[0][0]
    #return the rearranged data as a HyperSpy signal
    return hs.signals.Signal2D(np.concatenate((dat.inav[edge+1:,1:], dat.inav[0:edge,1:]), axis=1))

In [ ]:
dp = load_mib_scan_data(fname=filenames[-1], dtype='uint16', n_images=n_images)

In [ ]:
dp.plot()

In [ ]:
dp.save('/home/hremadmin/Documents/RefAu_EM17997_5_10umCLap_15cmL_150kX')

In [ ]:
dpeg = dp.sum((0,1))

In [ ]:
dpeg.change_dtype('float32')

In [ ]:
dpeg.save('/home/hremadmin/Documents/RefAu_EM17997_5_10umCLap_15cmL.tif')

In [ ]:
dpeg.plot(vmax=1e8)

In [ ]:
trace = pxm.roi.Line2DROI(x1=5,y1=5, x2=250,y2=250, linewidth=5)

In [ ]:
trace.add_widget(dpeg)

In [ ]:
line = trace(dpeg)

In [ ]:
line.plot()

In [ ]:
im = dp.sum((2,3))

In [ ]:
im=im.as_signal2D((0,1))

In [ ]:
im.plot()

In [ ]:
im.change_dtype('float32')

In [ ]:
im.save('/home/hremadmin/Documents/RefAu_EM17997_5_10umCLap_150kX.tif')

# Inspect data

In [ ]:
filenames = []
for root, dirs, files in os.walk("/epsic_eg/"):
    for file in files:
        if file.endswith(".hdr"):
             filenames.append(os.path.join(root, file))

In [ ]:
for fname in filenames:
    tstamp = os.path.dirname(fname)[-6:]
    #load data
    dp = pxm.load_mib(fname, scan_size=256)
    #crop center
    roi = hs.roi.RectangularROI(left=103, top=103, right=153, bottom=153)
    dpc = roi(dp, axes=dp.axes_manager.signal_axes)
    #perform alignment
    centers = dpc.get_direct_beam_position(sigma=3)
    shifts = centers.data - np.array((25,25))
    shifts = shifts.reshape(65025, 2)
    dp.align2D(shifts=shifts, crop=False, fill_value=0)
    #set calibrations
    dp.set_calibration(0.00889)
    dp.axes_manager.navigation_axes[0].scale = 5.033
    dp.axes_manager.navigation_axes[1].scale = 5.033
    dp.axes_manager.navigation_axes[0].units = 'nm'
    dp.axes_manager.navigation_axes[1].units = 'nm'
    #save data
    dp.save('epsic_eg/' + tstamp)
    #summed diffraction
    dpeg = dp.sum((0,1))
    dpeg.change_dtype('float32')
    dpeg.save('epsic_eg/' + tstamp + '.tif')
    #vbf image
    circ = hs.roi.CircleROI(cx=0, cy=0, r=10*0.00889)
    vbf = dp.get_virtual_image(circ)
    vbf = vbf.as_signal2D((0,1))
    vbf.change_dtype('float32')
    vbf.save('epsic_eg/' + tstamp + '.tif')
    #vdf image
    ann = hs.roi.CircleROI(cx=0, cy=0, r=100*0.00889, r_inner=50*0.00889)
    vdf = dp.get_virtual_image(ann)
    vdf = vdf.as_signal2D((0,1))
    vdf.change_dtype('float32')
    vdf.save('epsic_eg' + tstamp + '.tif')